In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Flatten
from keras.layers import Embedding
from keras.preprocessing import  sequence

In [3]:
def upload_accel(file_path):
    l=[]
    for i in os.listdir("WISDM Watch Accel and Gyro/Accel"):
        df=pd.read_csv(f"WISDM Watch Accel and Gyro/Accel/{i}", 
                       header=None, names=['subject id', 'activity', 'timestamp', 'x-accel','y-accel','z-accel'])
        df['z-accel'].replace(regex=True, inplace=True, to_replace=';', value='')
        df=df.drop(columns='timestamp')
        df=df.reset_index()
        l.append(df)
    accel=pd.concat(l)
    return accel

In [4]:
accel=upload_accel("WISDM Watch Accel and Gyro/Accel")

In [5]:
def upload_gyro(file_path):
    l=[]
    for i in os.listdir("WISDM Watch Accel and Gyro/Gyro"):
        df=pd.read_csv(f"WISDM Watch Accel and Gyro/Gyro/{i}", 
                       header=None, names=['subject id', 'activity', 'timestamp', 'x-gyro','y-gyro','z-gyro'])
        df['z-gyro'].replace(regex=True, inplace=True, to_replace=';', value='')
        df=df.drop(columns='timestamp')
        df=df.reset_index()
        l.append(df)
    accel=pd.concat(l)
    return accel

In [6]:
gyro=upload_gyro("WISDM Watch Accel and Gyro/Gyro")

In [7]:
gyro=gyro.reset_index()
accel=accel.reset_index()
data=pd.concat([accel, gyro],ignore_index=False, axis=1)
data=data.drop(columns=['level_0', 'index'])

In [8]:
data

,subject id,activity,x-accel,y-accel,z-accel,subject id,activity,x-gyro,y-gyro,z-gyro
0,1619,A,9.306112,-1.640178,-2.385074,1631.0,A,-2.019593,0.131862,0.8683165
1,1619,A,8.958953,-1.609053,-2.3108535,1631.0,A,-0.178816,0.320414,-0.4142619
2,1619,A,8.044368,-0.943465,-2.282123,1631.0,A,0.135437,-0.534993,-1.6489034
3,1619,A,8.848820,-0.177321,-2.5981576,1631.0,A,2.253182,-0.702240,-2.2358642
4,1619,A,8.173655,-0.117466,-2.459294,1631.0,A,-4.074488,-1.359508,-0.6358369
...,...,...,...,...,...,...,...,...,...,...
2432089,1605,S,4.962731,-8.595633,-7.9870567,NaN,NaN,NaN,NaN,NaN
2432090,1605,S,3.736900,-6.780828,-5.5401826,NaN,NaN,NaN,NaN,NaN
2432091,1605,S,2.015469,-6.929269,-3.7469256,NaN,NaN,NaN,NaN,NaN
2432092,1605,S,1.764078,-9.136723,-1.316811,NaN,NaN,NaN,NaN,NaN


In [9]:
data=data.dropna(axis=0)

In [10]:
activity=data['activity']

In [11]:
activity=activity.iloc[:,1]

In [12]:
data=data.drop(columns='activity')

In [13]:
data['activity']=activity

In [14]:
data

,subject id,x-accel,y-accel,z-accel,subject id,x-gyro,y-gyro,z-gyro,activity
0,1619,9.306112,-1.640178,-2.385074,1631.0,-2.019593,0.131862,0.8683165,A
1,1619,8.958953,-1.609053,-2.3108535,1631.0,-0.178816,0.320414,-0.4142619,A
2,1619,8.044368,-0.943465,-2.282123,1631.0,0.135437,-0.534993,-1.6489034,A
3,1619,8.848820,-0.177321,-2.5981576,1631.0,2.253182,-0.702240,-2.2358642,A
4,1619,8.173655,-0.117466,-2.459294,1631.0,-4.074488,-1.359508,-0.6358369,A
...,...,...,...,...,...,...,...,...,...
2430913,1605,0.983568,-8.286781,-1.9153614,1610.0,-0.508430,0.602261,0.061357632,S
2430914,1605,1.603666,-8.425645,0.2753332,1610.0,0.014615,0.945276,-0.16980475,S
2430915,1605,0.375441,-6.929269,1.0989386,1610.0,3.126252,1.933841,-0.83666027,S
2430916,1605,2.331504,-9.371354,7.7332716,1610.0,3.550228,2.166069,-1.326682,S


In [15]:
data['x-accel'].min()

-19.815012

In [16]:
# Normalize Data: Can normalize via z-score like below..must know standard deviation. Can also normalize via range
    #x-x.mean/(x.max-x.min)

def normalize(dataset):
    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu)/sigma

data

,subject id,x-accel,y-accel,z-accel,subject id,x-gyro,y-gyro,z-gyro,activity
0,1619,9.306112,-1.640178,-2.385074,1631.0,-2.019593,0.131862,0.8683165,A
1,1619,8.958953,-1.609053,-2.3108535,1631.0,-0.178816,0.320414,-0.4142619,A
2,1619,8.044368,-0.943465,-2.282123,1631.0,0.135437,-0.534993,-1.6489034,A
3,1619,8.848820,-0.177321,-2.5981576,1631.0,2.253182,-0.702240,-2.2358642,A
4,1619,8.173655,-0.117466,-2.459294,1631.0,-4.074488,-1.359508,-0.6358369,A
...,...,...,...,...,...,...,...,...,...
2430913,1605,0.983568,-8.286781,-1.9153614,1610.0,-0.508430,0.602261,0.061357632,S
2430914,1605,1.603666,-8.425645,0.2753332,1610.0,0.014615,0.945276,-0.16980475,S
2430915,1605,0.375441,-6.929269,1.0989386,1610.0,3.126252,1.933841,-0.83666027,S
2430916,1605,2.331504,-9.371354,7.7332716,1610.0,3.550228,2.166069,-1.326682,S


In [17]:
data.iloc[:,:2].apply(normalize)

,subject id,x-accel
0,0.112051,1.218865
1,0.112051,1.169653
2,0.112051,1.040004
3,0.112051,1.154041
4,0.112051,1.058331
...,...,...
2430913,-1.234902,0.039084
2430914,-1.234902,0.126987
2430915,-1.234902,-0.047123
2430916,-1.234902,0.230163


In [18]:
z_accel=data['z-accel'].astype(dtype='float64')

In [19]:
z_gyro=data['z-gyro'].astype(dtype='float64')

In [20]:
data=data.iloc[:,:6].astype('float64')

In [21]:
data=data.apply(normalize)

In [22]:
data['activity']=activity

In [23]:
data[data['activity']=='I']

,subject id,x-accel,y-accel,z-accel,subject id,x-gyro,activity
28826,0.112051,-0.328927,-0.163122,1.155305,1.267062,1.143661,I
28827,0.112051,-0.295667,-0.294398,1.158063,1.267062,1.658519,I
28828,0.112051,-0.193848,-0.243523,1.111633,1.267062,2.491082,I
28829,0.112051,-0.191133,-0.313931,1.090026,1.267062,2.974635,I
28830,0.112051,-0.191812,-0.319382,1.135997,1.267062,3.183109,I
...,...,...,...,...,...,...,...
2398461,-1.234902,-0.577259,-0.412359,0.959842,-0.753085,0.021886,I
2398462,-1.234902,-0.617308,-0.570435,0.874336,-0.753085,0.011230,I
2398463,-1.234902,-0.628168,-0.332867,0.805380,-0.753085,0.025217,I
2398464,-1.234902,-0.603732,-0.484130,0.778716,-0.753085,0.013228,I


In [24]:
data

,subject id,x-accel,y-accel,z-accel,subject id,x-gyro,activity
0,0.112051,1.218865,0.514351,-0.851502,1.267062,-1.244429,A
1,0.112051,1.169653,0.520257,-0.837251,1.267062,-0.093494,A
2,0.112051,1.040004,0.646536,-0.831734,1.267062,0.102991,A
3,0.112051,1.154041,0.791893,-0.892416,1.267062,1.427100,A
4,0.112051,1.058331,0.803250,-0.865753,1.267062,-2.529241,A
...,...,...,...,...,...,...,...
2430913,-1.234902,0.039084,-0.746681,-0.761313,-0.753085,-0.299583,S
2430914,-1.234902,0.126987,-0.773028,-0.340678,-0.753085,0.027448,S
2430915,-1.234902,-0.047123,-0.489126,-0.182538,-0.753085,1.972982,S
2430916,-1.234902,0.230163,-0.952453,1.091319,-0.753085,2.238070,S


In [25]:
data=data.drop(columns='activity')

In [26]:
labels=pd.get_dummies(activity) #For one hot encoding. Only functions on string variables

In [27]:
features=np.array(data)
labels=np.array(labels)

In [28]:
labels.shape

(2430918, 18)

In [29]:
features.shape

(2430918, 6)

In [30]:
m=len(data) #Converting data to np arrays and splitting manually...will test against sklearn train_test_split and 
#
crossval=round(3*m/4)
train_data=features[:crossval,:]
test_data=features[crossval:,:]
X_train=train_data
y_train=labels[:crossval,:]
X_test=test_data
y_test=labels[crossval:,:]

In [31]:
X_train.shape
X_train=np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))
X_test=np.reshape(X_test, (X_test.shape[0],1,X_test.shape[1]))

In [32]:
y_train.shape

(1823188, 18)

In [33]:
#Making the LSTM Model
model=Sequential()
model.add(LSTM(24,input_dim=6, return_sequences=True))
model.add(LSTM(12))
model.add(Dense(18, activation='sigmoid')) #Dense layer number must always 
#Match the number of potential outputs
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy']) #Can try categorical cross entropy,
#different batch sizes, different activation function for Dense

print(model.summary())

2022-12-22 17:13:28.826955: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-22 17:13:28.828672: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 24)          2976      
                                                                 
 lstm_1 (LSTM)               (None, 12)                1776      
                                                                 
 dense (Dense)               (None, 18)                234       
                                                                 
Total params: 4,986
Trainable params: 4,986
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
X_train=X_train.astype(dtype='float32')
y_train=y_train.astype(dtype='float32')

In [35]:
y_train.shape

(1823188, 18)

In [36]:
#Trying different batch sizes for multiple models with pow
#pow(2,5) 2^5=32
X_train=np.reshape(X_train, (1823188,1,6))

In [37]:
accel=upload_accel("WISDM Watch Accel and Gyro/Accel")
gyro=upload_gyro("WISDM Watch Accel and Gyro/Gyro")
gyro=gyro.reset_index()
accel=accel.reset_index()
data=pd.concat([accel, gyro],ignore_index=False, axis=1)
data=data.drop(columns=['level_0', 'index'])

In [38]:
data=data.dropna(axis=0)

In [39]:
activity=data['activity']
ID=data['subject id']

In [40]:
ID=pd.concat([ID.iloc[:,0], ID.iloc[:,1]])

In [41]:
ID

0          1619.0
1          1619.0
2          1619.0
3          1619.0
4          1619.0
            ...  
2430913    1610.0
2430914    1610.0
2430915    1610.0
2430916    1610.0
2430917    1610.0
Name: subject id, Length: 4861836, dtype: float64

In [42]:
bools=activity.iloc[:,0]==activity.iloc[:,1]

In [43]:
bools.isin([False]).any() #There are values that did not match in gyro and accel which must ahve thrown off my LSTM

True

In [44]:
#Goal: I need to find way to concatenate the accel and gyro data but while aligning both the subject and activity..

accel=accel.drop(columns=['level_0', 'index'])

In [45]:
#Starting with just accel data and going to build LSTM with each batch corresponding to specific subject.

accel.index=accel['subject id']

In [46]:
accel

,subject id,activity,x-accel,y-accel,z-accel
subject id,,,,,
1619,1619,A,9.306112,-1.640178,-2.385074
1619,1619,A,8.958953,-1.609053,-2.3108535
1619,1619,A,8.044368,-0.943465,-2.282123
1619,1619,A,8.848820,-0.177321,-2.5981576
1619,1619,A,8.173655,-0.117466,-2.459294
...,...,...,...,...,...
1605,1605,S,4.962731,-8.595633,-7.9870567
1605,1605,S,3.736900,-6.780828,-5.5401826
1605,1605,S,2.015469,-6.929269,-3.7469256


In [47]:
accel[accel.index==1619]

,subject id,activity,x-accel,y-accel,z-accel
subject id,,,,,
1619,1619,A,9.306112,-1.640178,-2.385074
1619,1619,A,8.958953,-1.609053,-2.3108535
1619,1619,A,8.044368,-0.943465,-2.282123
1619,1619,A,8.848820,-0.177321,-2.5981576
1619,1619,A,8.173655,-0.117466,-2.459294
...,...,...,...,...,...
1619,1619,S,-1.434276,-9.809194,1.7712604
1619,1619,S,-1.817349,-9.758916,0.94765496
1619,1619,S,-1.606659,-9.086145,0.06419453


In [48]:
accel=accel.sort_index()

In [49]:
accel

,subject id,activity,x-accel,y-accel,z-accel
subject id,,,,,
1600,1600,H,-5.475539,-7.633014,-0.38531685
1600,1600,L,-5.861006,-7.120655,2.3751976
1600,1600,L,-5.899313,-7.206846,2.3249195
1600,1600,L,-5.863400,-7.283461,2.2674587
1600,1600,L,-5.868188,-7.307403,2.260276
...,...,...,...,...,...
1635,1635,L,-1.667561,-2.072331,8.719383
1635,1635,L,-1.090559,1.128716,10.979509
1635,1635,L,-1.006762,-0.463428,10.373776


In [50]:
accel=accel.sort_values('activity')

In [54]:
from scipy.stats import zscore
accel['x-accel_n']=accel['x-accel'].transform(zscore)
accel['x-accel']

subject id
1635   -0.506523
1606    1.519869
1606    1.483956
1606    1.814356
1606    2.599654
          ...   
1624    4.382287
1624    2.770989
1624    3.178003
1624    2.940978
1610   -8.487145
Name: z-accel, Length: 2432094, dtype: float64

In [68]:
accel['y-accel_n']=accel['y-accel'].transform(zscore)
accel['z-accel_n']=accel['z-accel'].transform(zscore)
accel=accel.drop(columns=['x-accel','y-accel','z-accel'])


KeyError: 'y-accel'

In [69]:
accel

,subject id,activity,x-accel_n,y-accel_n,z-accel_n
subject id,,,,,
1635,1635,A,1.063793,-0.486441,-0.490635
1606,1606,A,1.025396,0.311211,-0.101529
1606,1606,A,1.674374,0.338468,-0.108425
1606,1606,A,1.041349,0.131318,-0.044982
1606,1606,A,1.286413,-0.409271,0.105810
...,...,...,...,...,...
1624,1624,S,0.336685,-0.288433,0.448110
1624,1624,S,0.259975,-0.624597,0.138710
1624,1624,S,0.425613,-0.442887,0.216864


In [70]:
accel.index

Int64Index([1635, 1606, 1606, 1606, 1606, 1606, 1606, 1606, 1606, 1606,
            ...
            1624, 1624, 1624, 1624, 1624, 1624, 1624, 1624, 1624, 1610],
           dtype='int64', name='subject id', length=2432094)

In [71]:
accel.loc[1600].values[:60000]

array([[1600, 'A', 1.4486359555815163, -0.3285229801944275,
        -1.1252385538981875],
       [1600, 'A', 1.8352394042409632, -0.28309540353080126,
        -0.8236540158801959],
       [1600, 'A', 2.3379257059815965, 0.6127371103143072,
        -0.2968004716115193],
       ...,
       [1600, 'R', -1.2453474257641175, 0.46836249598841484,
        -0.3603872328956214],
       [1600, 'R', -1.1574366639114269, 1.3764603796366592,
        -0.46106866719428813],
       [1600, 'R', -1.3298638553954854, 1.1038947299147939,
        0.17382203288969905]], dtype=object)

In [72]:
accel.index.unique()

Int64Index([1635, 1606, 1628, 1607, 1627, 1605, 1629, 1608, 1615, 1625, 1626,
            1602, 1631, 1603, 1601, 1632, 1633, 1604, 1630, 1613, 1619, 1618,
            1612, 1620, 1616, 1614, 1617, 1623, 1624, 1622, 1609, 1611, 1621,
            1610, 1634, 1600],
           dtype='int64', name='subject id')

In [73]:
#I want to write code to iterate through the dataframe.index and add 61000 values(approximately one subject) 
#from each unique index to an empty list so that I can create distinct batches of 61000 rows for each subject in the 
#model

l=[]
for i in accel.index.unique():
    arr=accel.loc[i].values[:60000]
    l.append(arr)

In [74]:
activity.iloc[:,0].unique().shape

(18,)

In [75]:
l

[array([[1635, 'A', 1.0637934194557874, -0.4864406362628823,
         -0.4906352143036975],
        [1635, 'A', 1.1109732862737702, 0.3030911584492328,
         -0.2377823027706234],
        [1635, 'A', 1.252512744958736, -1.1315126043665904,
         -0.03136238535156647],
        ...,
        [1635, 'R', -1.4278087848341396, -0.582207705610673,
         1.1324219049512403],
        [1635, 'R', -1.266582576370995, -0.5517711912980219,
         0.37386314154914646],
        [1635, 'R', -1.1650949392937575, 0.9468855684575792,
         -0.025184726246337132]], dtype=object),
 array([[1606, 'A', 1.0253960985833863, 0.3112113520083641,
         -0.10152906088525511],
        [1606, 'A', 1.6743738560697257, 0.33846795492857223,
         -0.10842506360848464],
        [1606, 'A', 1.0413489794267525, 0.13131801560232872,
         -0.044981957606642406],
        ...,
        [1606, 'R', -1.0685077172252155, 0.22035614175907922,
         -1.0191552543200084],
        [1606, 'R', -0.15172474062

In [76]:
l=np.asarray(l).reshape(36,60000,5)

In [77]:
labels=l[:,:, 1]

In [78]:
features=l[:,:,2:5]

In [79]:
features

array([[[1.0637934194557874, -0.4864406362628823, -0.4906352143036975],
        [1.1109732862737702, 0.3030911584492328, -0.2377823027706234],
        [1.252512744958736, -1.1315126043665904, -0.03136238535156647],
        ...,
        [-1.4278087848341396, -0.582207705610673, 1.1324219049512403],
        [-1.266582576370995, -0.5517711912980219, 0.37386314154914646],
        [-1.1650949392937575, 0.9468855684575792, -0.025184726246337132]],

       [[1.0253960985833863, 0.3112113520083641, -0.10152906088525511],
        [1.6743738560697257, 0.33846795492857223, -0.10842506360848464],
        [1.0413489794267525, 0.13131801560232872, -0.044981957606642406],
        ...,
        [-1.0685077172252155, 0.22035614175907922, -1.0191552543200084],
        [-0.15172474062321448, -1.2051623144381376, 2.1290933551036146],
        [-0.5654820347169736, -1.9797028760331803, 0.9609128825066876]],

       [[-1.4593327484736491, 0.9998655148434532, -0.5568366945121513],
        [-1.374137458068776, 

In [80]:
from keras.utils import np_utils

In [81]:
features.shape

(36, 60000, 3)

In [82]:
labels=labels.reshape(-1)

In [83]:
labels

array(['A', 'A', 'A', ..., 'R', 'R', 'R'], dtype=object)

In [84]:
## Transform the labels from String to Integer via LabelEncoder...Should be used on y data, not X

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
encoded_labels=le.fit_transform(labels)

In [85]:
encoded_labels=encoded_labels.reshape(36,60000,1)

In [86]:
features=features.astype('float32')
features

array([[[ 1.0637934 , -0.48644063, -0.49063522],
        [ 1.1109732 ,  0.30309117, -0.2377823 ],
        [ 1.2525127 , -1.1315126 , -0.03136238],
        ...,
        [-1.4278088 , -0.5822077 ,  1.1324219 ],
        [-1.2665826 , -0.55177116,  0.37386313],
        [-1.165095  ,  0.9468856 , -0.02518473]],

       [[ 1.0253961 ,  0.31121135, -0.10152906],
        [ 1.6743739 ,  0.33846796, -0.10842507],
        [ 1.0413489 ,  0.13131802, -0.04498196],
        ...,
        [-1.0685077 ,  0.22035614, -1.0191553 ],
        [-0.15172474, -1.2051623 ,  2.1290934 ],
        [-0.565482  , -1.9797028 ,  0.9609129 ]],

       [[-1.4593327 ,  0.99986553, -0.5568367 ],
        [-1.3741374 ,  0.96170634, -0.5172997 ],
        [-1.4925963 ,  0.8022554 , -0.49109495],
        ...,
        [-0.15836471,  2.7099311 ,  0.092593  ],
        [-0.42922467,  2.792155  ,  0.20384827],
        [-0.21538787,  2.736279  , -0.03383344]],

       ...,

       [[ 1.0450402 ,  0.15215796, -0.04207991],
        [ 1

In [88]:
features[0,:,2]

array([-0.49063522, -0.2377823 , -0.03136238, ...,  1.1324219 ,
        0.37386313, -0.02518473], dtype=float32)

In [89]:
#Train Test Split & data Normalize...cannot be done randomly with sklearn because LSTM learns from previous sample

X_train=features[:30]
y_train=encoded_labels[:30]
X_test=features[30:]
y_test=encoded_labels[30:]

In [90]:
classes=18

In [91]:
y_train

array([[[ 0],
        [ 0],
        [ 0],
        ...,
        [16],
        [16],
        [16]],

       [[ 0],
        [ 0],
        [ 0],
        ...,
        [16],
        [16],
        [16]],

       [[ 0],
        [ 0],
        [ 0],
        ...,
        [15],
        [15],
        [15]],

       ...,

       [[ 0],
        [ 0],
        [ 0],
        ...,
        [16],
        [16],
        [16]],

       [[ 0],
        [ 0],
        [ 0],
        ...,
        [15],
        [15],
        [15]],

       [[ 0],
        [ 0],
        [ 0],
        ...,
        [16],
        [16],
        [16]]])

In [92]:
y_train=np_utils.to_categorical(y_train, classes)

In [93]:
y_train=y_train.astype('float32')
y_test=y_test.astype('float32')

In [94]:
y_train=y_train.reshape(30*60000,18)

In [95]:
X_train=X_train.reshape(30,60000*3)

In [96]:
X_train=X_train.reshape(30,60000,3)
y_train=y_train.reshape(30,60000,18)

In [97]:
y_train=y_train.reshape(30*60000,18)

In [98]:
X_train=X_train.reshape(30*60000,1,X_train.shape[2])

In [99]:
y_train.shape

(1800000, 18)

In [100]:
X_train=X_train.reshape(30*60000,1,3)

In [103]:
X_train[0,:,0].mean()

1.0637934

In [104]:
y_train=y_train.reshape(30*60000,18)

In [105]:
y_train[0,0]

1.0

In [106]:
from keras.layers import Flatten
from keras.layers import GlobalAveragePooling1D

In [107]:
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D

In [111]:
#Model development

model_a=Sequential()
model_a.add(LSTM(256,input_dim=3,return_sequences=True, stateful=False))
model_a.add(LSTM(128, return_sequences=True))
model_a.add(LSTM(64, return_sequences=True))
model_a.add(LSTM(32))
model_a.add(Dense(units=18,activation='sigmoid')) #Dense layer number must always 
#Match the number of potential outputs
model_a.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy']) #Can try categorical cross entropy,
#different batch sizes, different activation function for Dense

print(model_a.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, None, 256)         266240    
                                                                 
 lstm_6 (LSTM)               (None, None, 128)         197120    
                                                                 
 lstm_7 (LSTM)               (None, None, 64)          49408     
                                                                 
 lstm_8 (LSTM)               (None, 32)                12416     
                                                                 
 dense_2 (Dense)             (None, 18)                594       
                                                                 
Total params: 525,778
Trainable params: 525,778
Non-trainable params: 0
_________________________________________________________________
None


In [112]:
model_a.fit(X_train, y_train, batch_size=1000, verbose=1, epochs=10)

Epoch 1/10


2022-12-09 13:18:53.661470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 13:18:54.063775: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 13:18:54.123555: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 13:18:54.187949: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 13:18:54.234761: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 13:18:54.637174: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-09 13:18:54.835364: I tensorflow/core/grappler/optimizers/cust

1800/1800 [==============================] - 48s 24ms/step - loss: 1.9133 - accuracy: 0.3635
Epoch 2/10
1800/1800 [==============================] - 44s 24ms/step - loss: 1.6334 - accuracy: 0.4479
Epoch 3/10
1800/1800 [==============================] - 44s 24ms/step - loss: 1.5940 - accuracy: 0.4596
Epoch 4/10
1800/1800 [==============================] - 43s 24ms/step - loss: 1.5754 - accuracy: 0.4657
Epoch 5/10
1800/1800 [==============================] - 44s 24ms/step - loss: 1.5629 - accuracy: 0.4706
Epoch 6/10
1800/1800 [==============================] - 44s 24ms/step - loss: 1.5540 - accuracy: 0.4732
Epoch 7/10
1800/1800 [==============================] - 44s 24ms/step - loss: 1.5473 - accuracy: 0.4752
Epoch 8/10
1800/1800 [==============================] - 44s 24ms/step - loss: 1.5418 - accuracy: 0.4767
Epoch 9/10
1800/1800 [==============================] - 44s 24ms/step - loss: 1.5379 - accuracy: 0.4778
Epoch 10/10
1800/1800 [==============================] - 44s 24ms/step - lo

In [ ]:
#My last hope to increase model accuracy is simply reducing the activity classes that the model has to decide from